<a href="https://colab.research.google.com/github/sahil3Vedi/Django-Recommender/blob/master/Collaborative_Filtering_Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# MOVIE RECOMMENDER USING COLLABORATIVE FILTERING

In [3]:
from google.colab import drive
import pandas as pd
import os
drive.mount('/content/drive')
from IPython.display import HTML, display
from numpy.random import randint

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [0]:
DATA_DIR = 'drive/My Drive/movielens'
moviescsv = "movie.csv"
creditscsv = "rating.csv"
MOV_PATH = os.path.join(DATA_DIR, moviescsv)
REV_PATH = os.path.join(DATA_DIR,creditscsv)
movdata = pd.read_csv(MOV_PATH)
revdata = pd.read_csv(REV_PATH)

In [0]:
list_of_movids = list(movdata["movieId"])
list_of_userIds = []
#Creating list of unique userIds:
current_user_Id = 0
for each_entry in revdata["userId"]:
  present_entry = each_entry
  if present_entry != current_user_Id:
    list_of_userIds.append(present_entry)
    current_user_Id = present_entry
user_row = list(revdata["userId"])

In [0]:
def normaliseArray(anyArray):
  min_list = min(anyArray)
  max_list = max(anyArray)
  temp = []
  for eachelem in anyArray:
    tempelem = round(-1 + (((eachelem - min_list)*2)/(max_list - min_list)),2)
    temp.append(tempelem)
  return temp

dataframeColumns = []
for each_Id in list_of_movids:
  dataframeColumns.append(str(each_Id))

# export Pivot Matrix for users startIndex to endIndex-1 inclusive
def exportPivotMatrix(startIndex,endIndex):
  
  dataframeIndices = []
  for every in range(startIndex, endIndex):
    dataframeIndices.append("User " + str(every))
  pivotDf = pd.DataFrame(0,index=dataframeIndices,columns=dataframeColumns)
  print("Creating Cosine Matrix for users " + str(startIndex) + " to " + str(endIndex-1) + " inclusive..")
  print("Total Elements: " + str(endIndex-startIndex) + " x 27278 = " + str((endIndex-startIndex)*27278) + " elements approx..")
  #get first index of the first user
  out = display(progress(0, startIndex-endIndex), display_id=True)
  
  userId = startIndex
  dfCounter = 0
  while userId<endIndex:
    
    #obtaining user rated list
    userRatedList = []
    userRatings = []
    currentIndex = user_row.index(userId)
    nextIndex = user_row.index(userId+1)
    for revCounter in range(currentIndex,nextIndex):
      userRatedList.append(str(revdata["movieId"][revCounter]))
      userRatings.append(float(revdata["rating"][revCounter]))
    userRating = normaliseArray(userRatings)
    
    #updating dataframe
    ratingCounter = 0
    rowref = "User " + str(userId)
    for eachId in userRatedList:
      movIndex = list_of_movids.index(int(eachId))
      pivotDf.iat[dfCounter,movIndex] = userRating[ratingCounter]
      ratingCounter += 1
    out.update(progress(userId+1-startIndex,endIndex-startIndex))
    userId+=1
    dfCounter+=1
  return pivotDf
    
    
    
      

In [57]:
pivot = exportPivotMatrix(1000,2000)
print(pivot)

Creating Cosine Matrix for users 1000 to 1999 inclusive..
Total Elements: 1000 x 27278 = 27278000 elements approx..


           1  2  3  4  5  6  ...  131252  131254  131256  131258  131260  131262
User 1000  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1001  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1002  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1003  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1004  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1005  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1006  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1007  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1008  1  0  0  0  0  0  ...       0       0       0       0       0       0
User 1009  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1010  0  0  0  0  0  0  ...       0       0       0       0       0       0
User 1011  0  0  0  0  0  0 